In [1]:
import sys
sys.path.append('/home/jquser')
from at_grid import *

In [2]:
# file_path='Data/'
# file_name='网格-交易A.xlsm' 
etf_dict=get_wg_info()

In [3]:
notice_df=get_wg_notice(etf_dict)
notice_df

,名称,代码,当前价,买入价,买入数,卖出价,卖出数,说明,当前格,末格,首格,格数
0,银行ETF,512800,0.978,0.947,800,0.990,800,,0.968,0.887,1.180,14
1,H股ETF,510900,1.201,1.172,900,1.224,800,,1.198,0.945,1.306,16
2,证券ETF,512880,0.879,0.851,1200,0.894,1100,,0.872,0.698,1.295,26
3,创业板,159915,1.363,1.308,800,1.380,700,,1.344,1.099,1.951,21
4,创业板50,159949,0.498,0.481,2100,0.515,2000,,0.498,0.433,0.785,21
5,500ETF,510500,5.023,4.727,200,5.045,200,,4.883,4.430,7.658,21
6,华宝油气,162411,0.520,0.504,1200,0.534,1100,,0.518,0.504,0.824,18
7,券商ETF,512000,0.837,0.807,1200,0.848,1200,,0.827,0.662,1.228,26


In [4]:
print get_wg_report(notice_df)

<table border="1" cellspacing="0" cellpadding="6" rules="rows" frame="void">
  <thead style="font-size:12px">
    <tr style="text-align: right;">
      <th align="left">名称</th>
      <th align="left">代码</th>
      <th>当前价</th>
      <th>买入价</th>
      <th>买入数</th>
      <th>卖出价</th>
      <th>卖出数</th>
    </tr>
  </thead>
  <tbody style="text-align:right;font-size:12px">
    <tr align="right">
      <td align="left">银行ETF</td>
      <td align="left">512800</td>
      <td>0.978</td>
      <td>0.947</td>
      <td>800</td>
      <td>0.990</td>
      <td>800</td>
    </tr>
    <tr align="right">
      <td align="left">H股ETF</td>
      <td align="left">510900</td>
      <td>1.201</td>
      <td>1.172</td>
      <td>900</td>
      <td>1.224</td>
      <td>800</td>
    </tr>
    <tr align="right">
      <td align="left">证券ETF</td>
      <td align="left">512880</td>
      <td>0.879</td>
      <td>0.851</td>
      <td>1200</td>
      <td>0.894</td>
      <td>1100</td>
    </tr>
    <tr align="

In [39]:
import numpy as np
import pandas as pd
import xlrd
import collections

import sys
sys.path.append('/home/jquser')
from at_grid import *

In [44]:
def get_wg_info(file_name):

    # 打开excel文件
    xlr_f=xlrd.open_workbook(file_name)

    
    # 获取第一个表
    table=xlr_f.sheet_by_index(0)

    # 获取标的列表
    etf_list=[str(item)[:6] for item in table.col_values(2)[4:] if item is not u'']

    # 标的信息有序字典 
    etf_dict={}
    etf_dict=collections.OrderedDict()
    
    # 遍历所有工作表
    for i,code in zip(range(1,len(etf_list)+2),etf_list):
        table=xlr_f.sheet_by_index(i)
        
        # 获取标的的网格价格、买入数量、卖出数量
        price_list=[round(item,3) for item in table.col_values(2)[6:] if item is not u'']
        buy_list=[int(item) for item in table.col_values(4)[6:] if item is not u'']
        sell_list=[int(item) for item in table.col_values(5)[6:] if item is not u'']
               
        # 保存到字典
        etf_dict.update({code:{'name':table.name,
                           'value':{'price': price_list, 'buy': buy_list, 'sell': sell_list}}})
    return etf_dict


def get_wg_notice(etf_dict):    
    data_ist=[] 
        
    # 当前时间
    end_date=pd.datetime.today()
    
    # 遍历标的信息字典
    for code in etf_dict.keys():
        # 获取收盘价格
        current_price=get_price(normalize_code(code),end_date=end_date,count=1,
                        frequency='daily',fields=['close'])['close'].iloc[-1]
        current_price=round(current_price,3)
        
        # 获取名称、当前价格、网格价格表、买入数量表、卖出数量表
        name=etf_dict.get(code).get('name')
        price_list=etf_dict.get(code).get('value').get('price')
        buy_list=etf_dict.get(code).get('value').get('buy')
        sell_list=etf_dict.get(code).get('value').get('sell')
        
        # 价格格位计数器
        i=0
        
        # 最接近的价格
        near_price=0
        
        # 遍历价格列表
        for item in price_list:
            # 以当前价格查找最接近的格位价格
            if current_price >= item :
                # 找到则跳出循环
                near_price=item
                break
            # 档位计数器递增
            i+= 1  
        
        count=len(price_list)
        # 判断格位情况
        if i==count:  
            # 当前价格低于最后一格，即破网、满仓
            # print name,code,current_price,0,0,price_list[i-2],sell_list[i-2],'破网'
            data_ist.append([name,code,current_price,'-','满仓',price_list[i-2],sell_list[i-2],
                             '破网','-',price_list[-1],price_list[0],count])
        elif i==0:
            # 当前价格高于0格，即清仓
            # print name,code,current_price,price_list[i+1],buy_list[i+1],'-','清仓','清仓'
            data_ist.append([name,code,current_price,price_list[i+1],buy_list[i+1],'-','清仓',
                             '清仓','-',price_list[-1],price_list[0],count])
        elif i==(count-1):
            # 当前价格在最后一格前
            # print name,code,current_price,price_list[i],buy_list[i],price_list[i-2],sell_list[i-2] 
            data_ist.append([name,code,current_price,price_list[i],buy_list[i],price_list[i-2],sell_list[i-2],
                             '',near_price,price_list[-1],price_list[0],count])
        else:  
            # 当前价格在其余格内
            # print name,code,current_price,price_list[i+1],buy_list[i+1],price_list[i-1],sell_list[i-1]        
            data_ist.append([name,code,current_price,price_list[i+1],buy_list[i+1],price_list[i-1],sell_list[i-1],
                             '',near_price,price_list[-1],price_list[0],count])
    
    notice_df=pd.DataFrame(data=data_ist,columns=['名称','代码','当前价','买入价','买入数',
                                           '卖出价','卖出数','说明','当前格','末格','首格','格数'])    
    return notice_df
 
def get_wg_report(notice_df):
    # 保留必要信息
    del notice_df['说明']
    del notice_df['当前格']
    del notice_df['末格']
    del notice_df['首格']
    del notice_df['格数']
    # 整理对齐方式
    notice_df['名称']='#'+notice_df['名称']
    notice_df['代码']='#'+notice_df['代码']
    notice_df.rename(columns={'名称':'$名称','代码':'$代码'},inplace=True)
    # 返回表格
    return notice_df.to_html(index=False,justify='right',
               classes='border="1" cellspacing="0" cellpadding="4" rules="rows" frame="void"')\
              .replace('<th></th>','<th>$代码</th>').replace('<tr>','<tr align="right">')\
            .replace('<th>$','<th align="left">').replace('<td>#','<td align="left">')
        
file_name='../../Data/网格-交易A.xlsm' 
etf_dict=get_wg_info(file_name)



In [45]:
notice_df=get_wg_notice(etf_dict)
notice_df

,名称,代码,当前价,买入价,买入数,卖出价,卖出数,说明,当前格,末格,首格,格数
0,银行ETF,512800,0.983,0.947,800,0.990,800,,0.968,0.887,1.180,14
1,H股ETF,510900,1.199,1.172,900,1.224,800,,1.198,0.945,1.306,16
2,证券ETF,512880,0.841,0.81,1200,0.851,1200,,0.83,0.698,1.295,26
3,创业板,159915,1.301,1.22,800,1.308,800,,1.263,1.099,1.951,21
4,创业板50,159949,0.477,0.448,2200,0.481,2200,,0.464,0.433,0.785,21
5,500ETF,510500,4.830,4.576,200,4.883,200,,4.727,4.430,7.658,21
6,华宝油气,162411,0.501,-,满仓,0.518,1200,破网,-,0.504,0.824,18
7,券商ETF,512000,0.801,0.768,1300,0.807,1300,,0.787,0.662,1.228,26


In [46]:
print get_wg_report(notice_df)

<table border="1" class="dataframe border="1" cellspacing="0" cellpadding="4" rules="rows" frame="void"">
  <thead>
    <tr style="text-align: right;">
      <th align="left">名称</th>
      <th align="left">代码</th>
      <th>当前价</th>
      <th>买入价</th>
      <th>买入数</th>
      <th>卖出价</th>
      <th>卖出数</th>
    </tr>
  </thead>
  <tbody>
    <tr align="right">
      <td align="left">银行ETF</td>
      <td align="left">512800</td>
      <td>0.983</td>
      <td>0.947</td>
      <td>800</td>
      <td>0.990</td>
      <td>800</td>
    </tr>
    <tr align="right">
      <td align="left">H股ETF</td>
      <td align="left">510900</td>
      <td>1.199</td>
      <td>1.172</td>
      <td>900</td>
      <td>1.224</td>
      <td>800</td>
    </tr>
    <tr align="right">
      <td align="left">证券ETF</td>
      <td align="left">512880</td>
      <td>0.841</td>
      <td>0.81</td>
      <td>1200</td>
      <td>0.851</td>
      <td>1200</td>
    </tr>
    <tr align="right">
      <td align="left">创业板<

<table border="1" class="dataframe border="1" cellspacing="0" cellpadding="4" rules="rows" frame="void"">
  <thead>
    <tr style="text-align: center;">
      <th>名称</th>
      <th>代码</th>
      <th>当前价</th>
      <th>买入价</th>
      <th>买入数</th>
      <th>卖出价</th>
      <th>卖出数</th>
    </tr>
  </thead>
  <tbody style="text-align:right;">
    <tr>
      <td>银行ETF</td>
      <td>512800</td>
      <td>0.972</td>
      <td>0.947</td>
      <td>800</td>
      <td>0.990</td>
      <td>800</td>
    </tr>
    <tr>
      <td>H股ETF</td>
      <td>510900</td>
      <td>1.196</td>
      <td>1.147</td>
      <td>900</td>
      <td>1.198</td>
      <td>900</td>
    </tr>
    <tr>
      <td>证券ETF</td>
      <td>512880</td>
      <td>0.840</td>
      <td>0.81</td>
      <td>1200</td>
      <td>0.851</td>
      <td>1200</td>
    </tr>
    <tr>
      <td>创业板</td>
      <td>159915</td>
      <td>1.299</td>
      <td>1.22</td>
      <td>800</td>
      <td>1.308</td>
      <td>800</td>
    </tr>
    <tr>
      <td>创业板50</td>
      <td>159949</td>
      <td>0.477</td>
      <td>0.448</td>
      <td>2200</td>
      <td>0.481</td>
      <td>2200</td>
    </tr>
    <tr>
      <td>500ETF</td>
      <td>510500</td>
      <td>4.815</td>
      <td>4.576</td>
      <td>200</td>
      <td>4.883</td>
      <td>200</td>
    </tr>
    <tr>
      <td>华宝油气</td>
      <td>162411</td>
      <td>0.500</td>
      <td>-</td>
      <td>满仓</td>
      <td>0.518</td>
      <td>1200</td>
    </tr>
    <tr>
      <td>券商ETF</td>
      <td>512000</td>
      <td>0.801</td>
      <td>0.768</td>
      <td>1300</td>
      <td>0.807</td>
      <td>1300</td>
    </tr>
  </tbody>
</table>